# Correr con GPU notebookk sobre transformers para traduccion de textos

This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Beyond text classification: Sequence-to-sequence learning

### A machine translation example

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2022-05-17 23:29:47--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.191.128, 173.194.192.128, 209.85.147.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.191.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.02s   

2022-05-17 23:29:47 (120 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [2]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [3]:
import random
print(random.choice(text_pairs))

('Give me a glass of milk.', '[start] Dame un vaso de leche. [end]')


In [4]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**Vectorizing the English and Spanish text pairs**

In [6]:
import tensorflow as tf
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = tf.keras.layers.TextVectorization( #se añadió el tf.keras para que jalara
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

**Preparing datasets for the translation task**

In [8]:
batch_size = 64

def format_dataset(eng, spa):
  '''
  funcion que genera un diccionario de palabrs en español
  e inglés
  '''
  eng = source_vectorization(eng)
  spa = target_vectorization(spa)
  return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [9]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


### Sequence-to-sequence learning with RNNs

**GRU-based encoder**

In [10]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english") #capa con nombre 'english'
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source) #primer capa embedding
#el texto original se hace vector
encoded_source = layers.Bidirectional( #capa bidereccional GRU
    layers.GRU(latent_dim), merge_mode="sum")(x)

la celda anterior es una Primera red de capas GRU en lugar de Long short term memory para velocidad de computo

**GRU-based decoder and the end-to-end model**

Para el decoder tambien se hace algo similar a la red anterior

In [11]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True) #capa GRU
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x) #drop_out que evita sobreajuste
target_next_step = layers.Dense(vocab_size, activation="softmax")(x) #capa densa 
#que arroja una distribución de las palabras en español
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

**Training our recurrent sequence-to-sequence model**

In [12]:
seq2seq_rnn.compile(
    optimizer="rmsprop", #rms prop
    loss="sparse_categorical_crossentropy", # función de muchas categorias
    metrics=["accuracy"]) #metrica de rendimiento
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 [==============================] - 311s 226ms/step - loss: 1.6368 - accuracy: 0.4155 - val_loss: 1.3238 - val_accuracy: 0.5036
Epoch 2/15
1302/1302 [==============================] - 283s 217ms/step - loss: 1.3164 - accuracy: 0.5256 - val_loss: 1.1621 - val_accuracy: 0.5673
Epoch 3/15
1302/1302 [==============================] - 284s 218ms/step - loss: 1.1739 - accuracy: 0.5758 - val_loss: 1.0826 - val_accuracy: 0.5986
Epoch 4/15
1302/1302 [==============================] - 283s 217ms/step - loss: 1.0821 - accuracy: 0.6083 - val_loss: 1.0436 - val_accuracy: 0.6179
Epoch 5/15
1302/1302 [==============================] - 282s 217ms/step - loss: 1.0347 - accuracy: 0.6327 - val_loss: 1.0278 - val_accuracy: 0.6279
Epoch 6/15
1302/1302 [==============================] - 286s 219ms/step - loss: 1.0031 - accuracy: 0.6517 - val_loss: 1.0246 - val_accuracy: 0.6337
Epoch 7/15
1302/1302 [==============================] - 282s 217ms/step - loss: 0.9835 - accuracy: 0.6651 - val_

en resumen el codificador toma la palabra entera y nomas se toman las primeras "n" palabras y predice la n+1, como se toma la secuencia entera de palabras, por eso se tarda en ejecutar

**Translating new sentences with our RNN encoder and decoder**

Traduccion con una red recurrente

In [13]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]" #token semilla
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence]) #muestra siguiente del token
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index] #añadir prediccion a frase
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]": #condicion de salida
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Tom doesn't understand any French.
[start] tom no sabe nada de francés [end]
-
Do you watch TV?
[start] televisión televisión [end]
-
I felt as happy as if I were still dreaming.
[start] me siento tan feliz como si yo estaba ahí [end]
-
My doctor told me to give up smoking.
[start] el médico me dijo que [UNK] que se [UNK] de fumar [end]
-
That serves you right.
[start] eso te ha hecho [end]
-
Let's not stay here long.
[start] no nos vamos a mucho tiempo aquí [end]
-
You don't want that, do you?
[start] no quieres que eso verdad [end]
-
I wanted to help you.
[start] quería ayudarte [end]
-
Tom did his best to protect Mary.
[start] tom hizo lo que pudo para mary [end]
-
Tom wants to help.
[start] tom quiere ayudar [end]
-
Make a decision and make it with the confidence that you are right.
[start] [UNK] una y lo que [UNK] y es lo que estás bien es el que cierto [end]
-
He frequently jumps from one topic to another while he is talking.
[start] Él se [UNK] a menudo de un poco más de lo qu

Lo que arroja la celda anterior son las traducciones que hace la red

las redes recurrentes tienen porblemas 
están limitadas en tamaño, secuencias muy largas las olvida 

### Sequence-to-sequence learning with Transformer

#### The Transformer decoder

Usa un Self atention que produce representaciones que tienen en ceunta el contexto de la secuencia de entrada

- este a diferencia de las RNN (recurrentes) mantiene una representación codificada de un formato de secuencia

**The `TransformerDecoder`**

In [16]:
class TransformerDecoder(layers.Layer):
  '''
  Lee secuencias hasta n e intenta predecir 
  a n+1
  '''
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

  def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

  def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

  def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### Putting it all together: A Transformer for machine translation

**PositionalEmbedding layer**

In [26]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

In [28]:
embed_dim = 256
dense_dim = 2048 #numero de neuronas
num_heads = 8 #numero de cabezas para el transformer

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
#encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
#x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
# encoder, decoder y outputs metidos como una sola entrada
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

Las lineas de codigo comentadas, no jalaron, porque dice que no se definieron

**Training the sequence-to-sequence Transformer**

In [29]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs = 15, validation_data=val_ds) #epochs = 30

Epoch 1/15
1302/1302 [==============================] - 79s 60ms/step - loss: 2.0643 - accuracy: 0.2865 - val_loss: 1.8413 - val_accuracy: 0.3093
Epoch 2/15
1302/1302 [==============================] - 76s 59ms/step - loss: 1.8776 - accuracy: 0.3145 - val_loss: 1.8029 - val_accuracy: 0.3189
Epoch 3/15
 893/1302 [===================>..........] - ETA: 21s - loss: 1.8618 - accuracy: 0.3190

KeyboardInterrupt: ignored

La mejora de lostiempos fue de un 20%

**Translating new sentences with our Transformer model**

In [30]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
I'll never forget visiting Paris last summer.
[start] tom no es un poco de la casa [end]
-
A man who breaks his promises cannot be trusted.
[start] tom no es un poco de la casa [end]
-
He doesn't know what to do.
[start] tom no es un poco de la casa [end]
-
Only girls' shoes are sold here.
[start] tom no es un poco de la casa [end]
-
I surprised myself.
[start] tom no es un poco de la casa [end]
-
I was living in Boston a few years ago.
[start] tom no es un poco de la casa [end]
-
The truth is that he was not fit for the job.
[start] tom no es un poco de la casa [end]
-
Tom asked me to speak to you.
[start] tom no es un poco de la casa [end]
-
I wonder if I should answer his letter.
[start] tom no es un poco de la casa [end]
-
We became very good friends.
[start] tom no es un poco de la casa [end]
-
Tom went to the gym three days a week.
[start] tom no es un poco de la casa [end]
-
I didn't know Tom was sick.
[start] tom no es un poco de la casa [end]
-
You are very brave.
[start] to

aquí podemos ver como las traduccciones mejoraron

## Summary